In [ ]:
import pandas as pd
from pandas.io import sql
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import json

from datetime import timedelta 
from sklearn.metrics import pairwise

from sklearn import metrics
from sklearn.preprocessing import StandardScaler

#Clustering Packages
from sklearn import cluster
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from matplotlib import cm

from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error

import scikitplot as skplt
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report
from yellowbrick.classifier import ClassPredictionError


Wildfire database source : https://www.kaggle.com/rtatman/188-million-us-wildfires

In [ ]:
# setting up connection using path of the SQLite Database
connection = sqlite3.connect('...')

query = '''
        SELECT *
        FROM Fires
        '''

df = sql.read_sql(query, con = connection)

In [ ]:
#importing the discovery and containted dates as datetime

query = '''
SELECT datetime(DISCOVERY_DATE) AS DISCOVERY_DATE,
datetime(CONT_DATE) AS CONT_DATE
FROM Fires;
'''
date = sql.read_sql(query, con = connection)
date['DISCOVERY_DATE'] = pd.to_datetime(date.DISCOVERY_DATE)
date['CONT_DATE']=pd.to_datetime(date.CONT_DATE)
df['DISCOVERY_DATE'] = date['DISCOVERY_DATE'].copy()
df['CONT_DATE'] = date['CONT_DATE'].copy()

#Looking at the fire_size feature

df['FIRE_SIZE'].describe()

In [ ]:
# Dropping columns that I didn't need for my model. 
# Primarily different Identifiers of Fires and Reporting Units
df.drop(['FPA_ID','SOURCE_SYSTEM_TYPE','SOURCE_SYSTEM','NWCG_REPORTING_AGENCY',
         'NWCG_REPORTING_UNIT_ID','NWCG_REPORTING_UNIT_NAME','SOURCE_REPORTING_UNIT',
         'SOURCE_REPORTING_UNIT_NAME','LOCAL_FIRE_REPORT_ID','LOCAL_INCIDENT_ID',
        'ICS_209_INCIDENT_NUMBER','ICS_209_NAME','MTBS_ID','Shape',
        'FIRE_NAME','MTBS_FIRE_NAME','FIRE_CODE','COMPLEX_NAME','OBJECTID'],axis=1,inplace=True)

In [ ]:
#Looking at the data types
df.info()

In [ ]:
#Resampling weekly to see how mean fire size varies over the time frame
s = df['FIRE_SIZE'].copy()
date = pd.concat([date,s], axis=1)
date = date.set_index('DISCOVERY_DATE').copy()
date.resample('W').mean().plot(figsize=(7,7),kind='line')

In [ ]:
#plotting all the fires
plt.figure(figsize=(10,10))
plt.scatter(x='LONGITUDE',y='LATITUDE',data=df)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

In [ ]:
#Investingating where null values are in the data
sns.heatmap(df.isnull(),cbar=False,cmap='crest')

In [ ]:
#Checking out correlations
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(),cbar=True,annot=True,cmap='Blues')

# Feature Engineering

In [ ]:
#Adding discovery month as a feature for predictions
DISCOVERY_MONTH = []
for i in df['DISCOVERY_DATE']:
    month = int(i.strftime("%Y-%m-%d")[5:7])
    DISCOVERY_MONTH.append(month)
df['DISCOVERY_MONTH'] = DISCOVERY_MONTH

#Removing null values - due to my DBSCAN algorithm only being capable of 
# about 30,000 fires being inputted, I could drop incomplete rows and still
# have more than enough rows for modelling. 
df.dropna(inplace=True)

df = df.astype({'DISCOVERY_TIME': 'int64'}).copy()
df = df.astype({'CONT_TIME': 'int64'}).copy()

df.head()

In [ ]:
#Cleaning the Time features - removing incomplete rows that were not
# 4 characters long
def cleaner(x):
    x=str(x)
    if len(x)<3:
        return '!'
    else:
        return x
    
df['DISCOVERY_TIME'] = df['DISCOVERY_TIME'].apply(cleaner).copy()
df['CONT_TIME'] = df['CONT_TIME'].apply(cleaner).copy()

df = df[df['DISCOVERY_TIME']!='!'].copy()
df = df[df['CONT_TIME']!='!'].copy()

def cleaner2(x):
    try:
        time = x[0]+x[1]+':'+x[2]+x[3]
    except:
        time = x[0]+':'+x[1]+x[2]
    return time

df['DISCOVERY_TIME'] = df['DISCOVERY_TIME'].apply(cleaner2).copy()
df['CONT_TIME'] = df['CONT_TIME'].apply(cleaner2).copy()

#Conversions of data type and creating one column combining 
# Date and Time

df['DISCOVERY_DATE_AND_TIME'] = df['DISCOVERY_DATE'].astype(str).str.cat(df['DISCOVERY_TIME'].astype(str),sep=" ")
df['DISCOVERY_DATE_AND_TIME'] = pd.to_datetime(df['DISCOVERY_DATE_AND_TIME']).copy()

df['CONT_DATE_AND_TIME'] = df['CONT_DATE'].astype(str).str.cat(df['CONT_TIME'].astype(str),sep=" ")
df['CONT_DATE_AND_TIME'] = pd.to_datetime(df['CONT_DATE_AND_TIME']).copy()

#Creating the Duration Feature. This is the difference between when the
#fire discovery date+time, and the contained date+time

df['DURATION'] = df['CONT_DATE_AND_TIME'] - df['DISCOVERY_DATE_AND_TIME']

def to_hours(x):
    return x.total_seconds()/3600

#conversion to hours
df['DURATION'] = df['DURATION'].apply(to_hours).copy()

The Dataframe is now ready to be used for clustering.

In [ ]:
# df.to_csv('wildfire_csv_withduration.csv')

# Clustering

The code below was used twice, as I carried out clustering at first using 30,000 of the most recent fires, as well as a second time using 30,000 of the older ones. 

In [ ]:
#Taking Long, Lat
#FOD_ID Unique identifier to make joining back to original DF easier
fire_locations = df[['FOD_ID','LATITUDE','LONGITUDE']].copy()
fire_locations.head()

In [ ]:
X = fire_locations.head(30000).copy()
#X = fire_locations.tail(30000).copy()

#Building the distance_matrix for DBSCAN

X['latitude_rad'] = X['LATITUDE']*np.pi/180
X['longitude_rad'] = X['LONGITUDE']*np.pi/180
distance_matrix = pairwise.haversine_distances(X.loc[:,
['latitude_rad', 'longitude_rad']])*6371

The two parameters that need to be adjusted for DBSCAN are epsilon and minimum samples. 
EPS is the maximum distance between two samples for one to be considered as in the neighborhood of the other.
Minimum Samples, as the name suggests, is the number of samples in a neighborhood for a point to be considered as a core point.
By trying different combinations of these I was able to find a suitable number of clusters.

In [ ]:
db = DBSCAN(eps=300, min_samples=2000, metric='precomputed')
y_db = db.fit_predict(distance_matrix)  
X['cluster'] = y_db
print(len(X.cluster.unique()))
X.cluster.value_counts()

In [ ]:
#plotting the clusters
plt.figure(figsize=(6, 6))
plt.scatter(X['LONGITUDE'], X['LATITUDE'], c=X['cluster'],
            cmap='spring', s=40)
plt.xlabel('Longitude', fontsize=24)
plt.ylabel('Latitude', fontsize=24)
plt.title('DBScan clustering (3 clusters)', fontsize=24)
plt.show()

In [ ]:
#merging the original df with the clusters df
df3 = pd.merge(df, X, on='FOD_ID', how='inner')
#Selecting one of the clusters
c2 = df3[df3['cluster']==-1].copy()

![3Clusters](/visuals/3clusters.png)

![5Clusters](/visuals/5clusters.png)

# Adding Weather Data

To add historic weather data I used the Virtual Crossing API - the response contained a large amount of weather data for the latitude,longitude, and date provided.
The code below was used several times for all the different clusters.

In [ ]:
c2['DISCOVERY_DATE'] = pd.to_datetime(c2['DISCOVERY_DATE'])
c22 = c2.reset_index(drop=True)

In [ ]:
# empty lists which I append the humidity and temperature to.
humidityc2 = []
tempsc2 = []

for i in range(0,7442):
    lat = round(c22['LATITUDE_x'][i],4)
    long = round(c22['LONGITUDE_x'][i],4)
    date = c22['DISCOVERY_DATE'][i].strftime("%Y-%m-%d")
    year = int(c22['DISCOVERY_DATE'][i].strftime("%Y-%m-%d")[0:4])
    month = int(c22['DISCOVERY_DATE'][i].strftime("%Y-%m-%d")[5:7])
    day = int(c22['DISCOVERY_DATE'][i].strftime("%Y-%m-%d")[8:10])
    
    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C{long}/{year}-{month}-{day}?unitGroup=metric&key={apikey}&include=obs'

    response = requests.request("GET",url)
    response_dict = json.loads(response.text)
    tempsc2.append(response_dict['days'][0]['temp'])
    humidityc2.append(response_dict['days'][0]['humidity'])
    

In [ ]:
c2['HUMIDITY'] = humidityc2
c2['TEMP'] = tempsc2
c2.dropna(inplace=True)

# Modelling

I carried out the same steps for each cluster:
Features were selected, and categorical variables were dummified.The data was standardized, and the target was selected. Following a training - validation split, using the standard 80-20 ratio, I fit a decision tree classifier on the training data. The mean cross validation score on the test set was then obtained. In the majority of cases the score was only marginally above baseline, however in the case of two cases the score achieved was far above baseline. For those two clusters I went further by applying grid search to optimize a random forest classifier. I then obtained a confusion matrix as well as other key metrics to evaluate my model. Finally, the feature importances were extracted.

In [ ]:
#Identifying the baseline accuracy for the model
c2['FIRE_SIZE_CLASS'].value_counts(normalize=True)

In [ ]:
X = c2[['LATITUDE_x','LONGITUDE_x','STAT_CAUSE_DESCR','DURATION',
       'OWNER_DESCR','COUNTY','STATE','DISCOVERY_DOY','CONT_DOY',
        'HUMIDITY','TEMP','DISCOVERY_MONTH']].copy()
X = pd.get_dummies(X,columns=['STAT_CAUSE_DESCR','OWNER_DESCR','DISCOVERY_MONTH',
                              'DISCOVERY_DOY','CONT_DOY','COUNTY','STATE'],drop_first=True).copy()
scaler = StandardScaler()
X = scaler.fit_transform(X)

y = c2['FIRE_SIZE_CLASS'].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    stratify=y, 
    test_size=0.2, 
    random_state=1)

model = DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(cross_val_score(model, X_test, y_test, cv=5).mean())

In [ ]:
model = RandomForestClassifier(
                           n_estimators=100,
                           random_state=1)
model.fit(X_train, y_train)
print(cross_val_score(model, X_test, y_test, cv=5).mean())

In [ ]:
param_grid = { 
    'n_estimators': [100, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

In [ ]:
skplt.metrics.plot_confusion_matrix(y_true, y_pred, figsize=(6,6))
plt.show()

In [ ]:
y_true = y_test
y_pred = model.predict(X_test)

target_names = ['Fire Size A', 'Fire Size B']
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
classes = ['A','B']

visualizer = ClassPredictionError(
    RandomForestClassifier(random_state=1, n_estimators=100), classes=classes
)

# Fit the training data to the visualizer
visualizer.fit(X_train, y_train)

# Evaluate the model on the test data
visualizer.score(X_test, y_test)

# Draw visualization
visualizer.show()

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_true, y_pred, labels=None, weights=None)

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_corrcoef(y_true, y_pred)

In [ ]:
probabilities_test = model.predict_proba(X_test)


cmap = ListedColormap(sns.color_palette("husl", len(model.classes_)))

skplt.metrics.plot_precision_recall(y_test, probabilities_test, cmap=cmap,figsize=(7,7))
plt.show()

In [ ]:
skplt.metrics.plot_roc(y_test, probabilities_test, cmap=cmap,figsize=(7,7))
plt.show()

In [ ]:
X = c2[['LATITUDE_x','LONGITUDE_x','STAT_CAUSE_DESCR','DURATION',
       'OWNER_DESCR','COUNTY','STATE','DISCOVERY_DOY','CONT_DOY',
        'HUMIDITY','TEMP','DISCOVERY_MONTH']].copy()
X = pd.get_dummies(X,columns=['STAT_CAUSE_DESCR','OWNER_DESCR','DISCOVERY_MONTH',
                              'DISCOVERY_DOY','CONT_DOY','COUNTY','STATE'],drop_first=True).copy()


In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(6).plot(kind='barh')
plt.title("Top 6 Feature Importances")
plt.show()

### Principle Comoponent Analysis

With a mean CV score noticeably higher than baseline, PCA proved to be 
an effective way of reducing dimensionality. With that being said,
being unable to extract feature importances made it not particularly 
useful for acheiving my goal, although it was nice to see 3 components retain the original signal in the data so well. 

In [ ]:
# 5 components captured over 97% of the variance.
c2_pca = PCA(n_components=5)
c2_pca.fit(X)

In [ ]:
print('Explained Variance Ratios:')
c2_pca.explained_variance_ratio_

In [ ]:
#Verification step
c2_pcs = c2_pca.transform(X)
print(np.allclose(np.corrcoef(c2_pcs.T), np.eye(len(c2_pcs.T))))

In [ ]:
c2_pcs = pd.DataFrame(c2_pca.transform(X), 
                        columns=[f'PC_{i+1}' for i in range(c2_pca.n_components_)])
c2_pcs.head()

In [ ]:
#Random Forest on PCA, seeing how well the model performs with 3 components
X = c2_pcs[['PC_1','PC_2','PC_3']]

scaler = StandardScaler()
scaler.fit_transform(X)

y = c2['FIRE_SIZE_CLASS'].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    stratify=y, 
    test_size=0.2, 
    random_state=1)

model = RandomForestClassifier(
                           n_estimators=100,
                           random_state=1)
model.fit(X_train, y_train)

print(cross_val_score(model, X_test, y_test, cv=5).mean())